<a href="https://colab.research.google.com/github/syous154/Project/blob/main/Dacon_1_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import pandas as pd
import numpy as np
import os
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models
from torchvision import transforms

from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score
!pip install rembg
from rembg.bg import remove as remove_bg

import warnings
warnings.filterwarnings(action='ignore') 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 78.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 63.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 KB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 KB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 MB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 KB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 KB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 KB 35.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)


cuda


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install efficientnet_pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=b3c4d9ad8a9f38242d6dd1625d93a937636ca10366cfc2beac91de11d54b3673
  Stored in directory: /root/.cache/pip/wheels/84/b9/90/25a0195cf95fb5533db96f1c77ea3f296b7cc86ae8ae48e3dc
Successfully built efficientnet_pytorch


In [ ]:
from sklearn.model_selection import train_test_split

from efficientnet_pytorch import EfficientNet

ModuleNotFoundError: ignored

In [ ]:

CFG = {
    'IMG_SIZE':224,
    'EPOCHS': 20,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':32,
    'SEED':41
}

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정"

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/dataset/train.csv')

In [ ]:
def get_labels(df):
    return df.iloc[:,2:].values

train, test, train_labels, test_labels = train_test_split(df, get_labels(df),
                                                     test_size=0.2, random_state=42)
val, test, val_labels, test_labels = train_test_split(test, get_labels(test),
                                                     test_size=0.5, random_state=42)
test['id']

20751    TRAIN_20751
19377    TRAIN_19377
6656     TRAIN_06656
22067    TRAIN_22067
9948     TRAIN_09948
            ...     
9112     TRAIN_09112
30418    TRAIN_30418
30968    TRAIN_30968
11137    TRAIN_11137
25666    TRAIN_25666
Name: id, Length: 3300, dtype: object

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = torch.FloatTensor(self.label_list[index])
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

class TestDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        image = remove_bg(image) ##바뀐부분
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = torch.FloatTensor(self.label_list[index])
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

        

In [ ]:
train['id'].values

array(['TRAIN_05196', 'TRAIN_06547', 'TRAIN_10943', ..., 'TRAIN_11284',
       'TRAIN_00860', 'TRAIN_15795'], dtype=object)

In [ ]:
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

train_dataset = CustomDataset('/content/drive/MyDrive/Colab_Notebooks/dataset/train/'+train['id'].values+'.jpg', train_labels, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)


val_dataset = CustomDataset('/content/drive/MyDrive/Colab_Notebooks/dataset/train/'+val['id'].values+'.jpg', val_labels, test_transform)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
class Model(nn.Module):
    def __init__(self, num_classes=10):
        super(Model, self).__init__()
        self.backbone = models.efficientnet_b7(pretrained=True)
        self.dropout = torch.nn.Dropout(0.5)
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.dropout(x)
        x = F.sigmoid(self.classifier(x))
        return x

In [ ]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.BCELoss().to(device)
    
    best_val_acc = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_acc = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val ACC : [{_val_acc:.5f}]')
        
        if scheduler is not None:
            scheduler.step(_val_acc)
            
        if best_val_acc < _val_acc:
            best_val_acc = _val_acc
            best_model = model
    
    return best_model

def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    val_acc = []
    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            probs = model(imgs)
            
            loss = criterion(probs, labels)
            
            probs  = probs.cpu().detach().numpy()
            labels = labels.cpu().detach().numpy()
            preds = probs > 0.5
            batch_acc = (labels == preds).mean()
            
            val_acc.append(batch_acc)
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_acc = np.mean(val_acc)
    
    return _val_loss, _val_acc

In [ ]:
model = Model()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)


In [ ]:
infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

  0%|          | 0/825 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]

Epoch [1], Train Loss : [0.21145] Val Loss : [0.07611] Val ACC : [0.97114]


  0%|          | 0/825 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]

...

  0%|          | 0/104 [00:00<?, ?it/s]

Epoch [19], Train Loss : [0.00138] Val Loss : [0.06494] Val ACC : [0.99216]
Epoch 00019: reducing learning rate of group 0 to 3.7500e-05.


  0%|          | 0/825 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]

Epoch [20], Train Loss : [0.00099] Val Loss : [0.05841] Val ACC : [0.99318]


In [ ]:
#test = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/dataset/test.csv')
#test_dataset = TestDataset('/content/drive/MyDrive/Colab_Notebooks/dataset/train/'+test['id'].values+'.jpg', None, test_transform)
#test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)
#testing_dataset = TestDataset('/content/drive/MyDrive/Colab_Notebooks/dataset/train/'+testing['id'].values+'.jpg', None, test_transform)
#testing_loader = DataLoader(testing_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
test_dataset = TestDataset('/content/drive/MyDrive/Colab_Notebooks/dataset/train/'+test['id'].values+'.jpg', None, test_transform)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

In [ ]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    predictions = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            probs = model(imgs)

            probs  = probs.cpu().detach().numpy()
            preds = probs > 0.5
            preds = preds.astype(int)
            predictions += preds.tolist()
    return predictions
preds = inference(model, test_loader, device)
#predict = inference(model, testing_loader, device)

  0%|          | 0/104 [00:00<?, ?it/s]


  0%|                                               | 0.00/176M [00:00<?, ?B/s]
  0%|                                       | 410k/176M [00:00<00:47, 3.74MB/s]
  1%|▍                                     | 2.10M/176M [00:00<00:15, 11.2MB/s]
  2%|▉                                     | 4.37M/176M [00:00<00:10, 16.3MB/s]
  5%|██                                    | 9.59M/176M [00:00<00:05, 30.3MB/s]
 10%|███▋                                  | 16.8M/176M [00:00<00:03, 45.2MB/s]
 14%|█████▏                                | 24.2M/176M [00:00<00:02, 54.8MB/s]
 17%|██████▌                               | 30.2M/176M [00:00<00:02, 56.7MB/s]
 21%|████████                              | 37.3M/176M [00:00<00:02, 61.2MB/s]
 25%|█████████▌                            | 44.1M/176M [00:00<00:02, 63.2MB/s]
 29%|██████████▉                           | 50.9M/176M [00:01<00:01, 64.8MB/s]
 33%|████████████▍                         | 57.8M/176M [00:01<00:01, 66.2MB/s]
 37%|█████████████▉                    

In [ ]:
result=[]
for idx in range(len(preds)):
  if (test_labels[idx] == preds[idx]).all():
    result.append(1)
  else:
    result.append(0)
result = np.array(result)

accuracy = result.sum() / len(result)
print('Accuracy : ',accuracy)

Accuracy :  0.0012121212121212121


In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/dataset/sample_submission.csv')

submit.iloc[:,1:] = preds
submit.head()

In [ ]:
submit.to_csv('/content/drive/MyDrive/Colab_Notebooks/dataset/efficientb7_tr3_bf.csv', index=False)

In [ ]:
df_re = pd.read_csv('./train.csv') 

In [ ]:
train, test, train_labels, test_labels = train_test_split(df_re, get_labels(df_re),
                                                     test_size=0.2, random_state=42)

train1, train2, train1_labels, train2_labels = train_test_split(train, get_labels(train),
                                                              test_size=0.2, random_state=42)

val, test, val_labels, test_labels = train_test_split(test, get_labels(test),
                                                     test_size=0.5, random_state=42)

In [ ]:
train_dataset = CustomDataset(train['img_path'].values, train_labels, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

train1_dataset = CustomDataset(train1['img_path'].values, train1_labels, train_transform)
train1_loader = DataLoader(train1_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

train2_dataset = CustomDataset(train2['img_path'].values, train2_labels, train_transform)
train2_loader = DataLoader(train2_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val['img_path'].values, val_labels, test_transform)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
def train_trans(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.BCELoss().to(device)
    
    best_val_acc = 0
    best_model = None
    
    for epoch in range(1, 10+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_acc = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val ACC : [{_val_acc:.5f}]')
        
        if scheduler is not None:
            scheduler.step(_val_acc)
            
        if best_val_acc < _val_acc:
            best_val_acc = _val_acc
            best_model = model
    
    return best_model

In [ ]:
infer_model = train_trans(model, optimizer, train1_loader, val_loader, scheduler, device)

In [ ]:
infer_model = train_trans(model, optimizer, train1_loader, val_loader, scheduler, device)

In [ ]:
result=[]
for idx in range(len(predict)):
  if (testing_labels[idx] == predict[idx]).all():
    result.append(1)
  else:
    result.append(0)
result = np.array(result)

accuracy = result.sum() / len(result)
print('Accuracy : ',accuracy)

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/dataset/sample_submission.csv')

submit.iloc[:,1:] = preds
submit.head()

In [ ]:
def inference(model, device):
    model.to(device)
    model.eval()
    predictions = []
    with torch.no_grad():
        for name in test['id'].values:
            imgs = cv2.imread('/content/drive/MyDrive/Colab_Notebooks/dataset/test/' + str(name) +'.jpg')
            imgs = remove_bg(imgs)
            imgs = cv2.cvtColor(imgs, cv2.COLOR_BGR2RGB)
            
            imgs = imgs.float().to(device)
            
            probs = model(imgs)

            probs  = probs.cpu().detach().numpy()
            preds = probs > 0.5
            preds = preds.astype(int)
            predictions += preds.tolist()
    return predictions
preds = inference(model, device)
predict = inference(model ,device)